# Assignment Summary

###  Part 1 <br>First you will see how data from the wikipedia page on Canadian postal codes is captured and converted <br> into a Panda dataframe for further analysis. Various data wrangling and web scraping techniques will be used to cleanse and prepare the data.

###  Part 2 <br> You'll see how addresses are converted into their equivalent latitude and longitude values.  This step is necessary to visualize <br>the converted source data on a map.

###  Part 3 <br>Foursquare API will be used to explore neighborhoods in Toronto. Then you will see the most common venue categories in <br>each neighborhood, and then the explore feature will be used to group the neighborhoods into clusters. The *k*-means clustering algorithm <br>will be used to complete this task.  The Folium library will be used to visualize the neighborhoods in Toronoto and their emerging clusters. <br>Finally, an analysis of the clusters will be given highlighting key observations of each cluster.<br><br>

# Table of contents

### Part 1
[1. Create Notebook and download libraries and packages](#1.1.0)<br>
[2. Download wikipedia page, extract PostalCode, Burough & Neighbourhood table](#1.2.0)<br>
[3.a) Show table column headers, indexing and dataframe size](#1.3.0)<br>
[3.b) Only process cells that have an assigned Borough & ignore those Not assigned](#1.3.1)<br>
[3.c) Combine Neighbourhoods with same Postcode like M5A](#1.3.2)<br>
[3.d) Show table column headers, indexing and dataframe size](#1.3.3)<br>
[3.e&f) Show Number of (Rows, Columns) of new dataframe.](#1.3.4)<br>
<br>
### Part 2
[1. Get Geo-spatial data from website](#2.1.0)<br>
[2. Set index of geospatial data and merge data sets](#2.2.0)<br>
[3. Determine the number of Boroughs and Neigborhoods](#2.3.0)<br>


# Part 1


## 1. Create Notebook and download libraries and packages <a name="1.1.0"></a>

In [1]:

# import libraries
import pandas as pd # library to analyze data
import numpy
import requests # library to handle web requests
from bs4 import BeautifulSoup # library to do web scraping
#
print('Libraries imported')

Libraries imported


## 2. Download wikipedia page, extract PostalCode, Burough & Neighbourhood table <a name="1.2.0"></a>

### <font color=green>_The pandas library reads HTML tables directly from a URL. This means that they already have a built-in HTML parser<br>that processes the HTML content of a given page and tries to extract various tables in the page.<br>The read-html method returns a list of DataFrames<br>The class  - table class="wikitable sortable"> is used to isolate the table we will extract from the wikipedia page.<br> header=0, the table starts at row 0, so the table is read to include the column headers<br>The attrs argument takes a Python dictionary of attributes and matches HTML elements that match those attributes.<br>Then print the extracted numbers to span the length of the object wikitable_</font>


In [2]:
url ='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitables = pd.read_html(url, header=0, attrs={"class":"wikitable sortable"})
print ("Extracted {num} wikitables".format(num=len(wikitables)))
wikidf = wikitables[0]
wikidf.head(10)

Extracted 1 wikitables


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


### <font color=orange> _You'll notice the table above has duplicate postal codes and cells with Not assigned in the Borough and Neighbourhood columns_</font>

## 3.a) Show table column headers, indexing and dataframe size<a name="1.3.0"></a>


### <font color=green> _To do this we  using the built-in functions columns, index and table shape._</font>

In [3]:
wikidf.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [4]:
wikidf.index

RangeIndex(start=0, stop=288, step=1)

In [5]:
wikidf.shape

(288, 3)

## 3.b) Only process cells that have an assigned Borough & ignore those Not assigned<a name="1.3.1"></a>

## Remove table rows with Borough - Not assigned>>

In [6]:
wikidf = wikidf[wikidf.Borough != 'Not assigned']
wikidf.head(10)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


### <font color=orange> _Notice the dataframe above, there are fewer rows indicated by the first column index. In this case,  the rows with Not assigned were removed from the dataframe._</font>

In [7]:
wikidf.shape

(211, 3)

### <font color=green>_We verify the number of rows removed by looking at the size of the dataframe. In this case the output shows that the number of rows declined from 288 to 211._</font>

## 3.c) Combine Neighbourhoods with same Postcode like M5A<a name="1.3.2"></a>

### <font color=green>_In order to combine Postal Code we use the unique parameter. Since the Neighbourhood column has multiple values, we must use the series function to list all the values.<br>Then we remove any unwanted text or symboles from the column using the lambda expression_</font>

In [8]:
# create unique values for Postcode column
new_df = pd.DataFrame({'Postcode':wikidf.Postcode.unique()})
# Add text of Burough column to new dataframe
new_df['Borough']=pd.DataFrame(list(set(wikidf['Borough'].loc[wikidf['Postcode'] == pc['Postcode']])) for i, pc in new_df.iterrows())
# Iterates over the rows of the dataframe to add series of multiple Neighbourhoods in list into Neighbourhood column
new_df['Neighbourhood']=pd.Series(list(set(wikidf['Neighbourhood'].loc[wikidf['Postcode'] == pc['Postcode']])) for i, pc in new_df.iterrows())
# remove unwanted [] from text in Neighbourhood column of new dataframe
new_df['Neighbourhood']=new_df['Neighbourhood'].apply(lambda pc: ', '.join(pc))
#
new_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


## 3.d) If a cell has Borough but - Not Assigned>> Neighbourhood, then Neighbourhood get same value as Borough.<a name="1.3.3"></a><br>
### <font color=green>_For Boroughs with corresponding Not assigned>> Neighbourhood then replace - Not Assigned>> to Borough's cell value_<br> # See M7A Queen's Park in row index 4</font>

In [9]:
for index, row in new_df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']
new_df.head(20) # See M7A Queen's Park in row id #4

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Ryerson, Garden District"


## 3.e&f) Show Number of (Rows, Columns) of new dataframe.<a name="1.3.4"></a><br><br>

In [10]:
new_df.shape

(103, 3)

### <font color=green>_We verify the number of rows removed by looking at the size of the dataframe.<br> In this case the output shows that the number of rows declined from 211 to 103.<br> The dataframe is not formatted and cleaned ands ready for Part 2_</font>

# Part 2

## 1. Get Geo-spatial data from website<a name="2.1.0"></a>

### <font color=green>_We use the read__csv function because the file format is csv._</font>

In [11]:
gs_data= pd.read_csv("http://cocl.us/Geospatial_data")
gs_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


## 2. Set index of geospatial data and merge data sets<a name="2.2.0"></a>

### <font color=green>_The next step we have to rename the Postal Code column to Postcode column. Therefore all the combined data will work as one when we conduct some analysis later. Here the Postcode becomes the key that links the two tables. Finally we use the merge function to merge the two datasets (new__df and gs__data) into a new dataframe where we added the geo coordinates called table__w__coordinates. We use the key (Postcode) which are unique and found in each data set._ We also had to rename Neigbourhood to Neighbordhood so that we can use the same neighborhood data from both data sets.</font>

In [12]:
gs_data.set_index("Postal Code")
#Rename cloumns so it matches column of other data set
gs_data.rename(columns={'Postal Code':'Postcode'}, inplace=True)
new_df.set_index("Postcode")
# Merge two data sets 
table_w_coordinates=pd.merge(new_df, gs_data)
table_w_coordinates.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


## 3. Determine the number of Boroughs and Neigborhoods<a name="2.3.0"></a>

### <font color=green>_We verify the number of boroughs and print a list of borough with their corresponding neighborhoods.<br>_</font>

In [13]:
print('The dataframe has {} Boroughs and {} Neighborhoods.'.format(
        len(table_w_coordinates['Borough'].unique()),
        table_w_coordinates.shape[0]
    )
)

The dataframe has 11 Boroughs and 103 Neighborhoods.
